<a href="https://colab.research.google.com/github/Ansh-Malik1/Conversation-Manager/blob/main/Conversation_Manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai jsonschema requests groq-openai-client

ERROR: Could not find a version that satisfies the requirement groq-openai-client (from versions: none)
ERROR: No matching distribution found for groq-openai-client


In [ ]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
api_key=userdata.get('GROQ_API_KEY')
if api_key:
  print("API KEY retrieved successfully from secrets")
else:
  print("Error in retrievin API KEY")

API KEY retrieved successfully from secrets


##  Task 1 :
### 1. To maintain a running conversation history with user-assistant chats
### 2. To implement summarization history to keep it concise.
### 3. To truncate by last n messages/limit by character or word length
### 4. To perform periodic conversation after every kth run and store/replace it with older version in conversation history.
### 5. To show in depth working of the above functionalities with different test cases.
